# Interchange Data Processing
The purpose of this notebook is to process the interchange data into a format that can be uploaded to the Neo4j database.

In [3]:
import pandas as pd
import numpy as np
from common_processing import clean_station

In [4]:
df_raw = pd.read_csv(r"../data/raw/interchange_data_raw.csv", header=0, encoding="windows-1252")
df_raw.head(10)

,STATION NAME:,"DETAILS OF MAXIMUM INTERCHANGE VALUES BETWEEN LINES (where appropriate) ï¿½ all values apply in both directions. However, in some cases there may be lower values according to station layout."
0,Abbey Road,NaN
1,Acton Central,NaN
2,Acton Town,District <> Piccadilly line: 2 minutes
3,Aldgate,Connections between Circle and Metropolitan li...
4,Aldgate East,Connections between District and Hammersmith &...
5,All Saints,NaN
6,Alperton,NaN
7,Amersham,Connections between Metropolitan and Chiltern ...
8,Anerley,Connections between London Overground and Sout...
9,NaN,NaN


In [5]:
new_col_names = ['station', 'interchange']
df_raw.columns = new_col_names
df_raw.head(5)

,station,interchange
0,Abbey Road,NaN
1,Acton Central,NaN
2,Acton Town,District <> Piccadilly line: 2 minutes
3,Aldgate,Connections between Circle and Metropolitan li...
4,Aldgate East,Connections between District and Hammersmith &...


In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 632 entries, 0 to 631
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   station      545 non-null    object
 1   interchange  345 non-null    object
dtypes: object(2)
memory usage: 10.0+ KB


In [7]:
df_no_nan_rows = df_raw.dropna(axis=0, how='any').reset_index(drop=True)
df_no_nan_rows.head(10)

,station,interchange
0,Acton Town,District <> Piccadilly line: 2 minutes
1,Aldgate,Connections between Circle and Metropolitan li...
2,Aldgate East,Connections between District and Hammersmith &...
3,Amersham,Connections between Metropolitan and Chiltern ...
4,Anerley,Connections between London Overground and Sout...
5,Baker Street,Bakerloo to Circle: 8 minutes
6,Baker Street,Bakerloo to Hammersmith & City: 8 minutes
7,Baker Street,Bakerloo to Jubilee: 2 minutes
8,Baker Street,Bakerloo to Metropolitan: 3 minutes
9,Baker Street,Circle to Hammersmith & City: 2 minutes


In [8]:
# Common station processing
df_clean_stations = df_no_nan_rows.assign(
    station = df_no_nan_rows["station"].apply(clean_station)
)

# Additional station processing
df_clean_stations["station"] = df_clean_stations["station"].apply(
    lambda x: x if x.find(",") == -1 else x[:x.find(",")]
)

In [9]:
# Correcting station names
df_clean_stations.loc[df_clean_stations["station"] == "Edgware Road Circle", "station" ] = "Edgware Road"
df_clean_stations.loc[df_clean_stations["station"] == "Hammersmith District & Piccadilly Lines", "station" ] = "Hammersmith"
df_clean_stations.loc[df_clean_stations["station"] == "Kensington Olympia", "station" ] = "Kensington"
df_clean_stations.loc[df_clean_stations["station"] == "Shepherds Bush Market", "station" ] = "Shepherds Bush"
df_clean_stations.loc[df_clean_stations["station"] == "St Jamess Park", "station" ] = "St James Park"

In [10]:
# Writing to file so I can review
df_clean_stations.to_csv(r"../data/raw/interchange_data_raw_v2.csv")

We now need to ensure that the station names align with those stored in the cleaned Station data.

In [11]:
df_stations_ref = (pd.read_csv(r"../data/processed/stations_clean.csv"))
stations_ref = set(df_stations_ref["Station"])
interchange_stations = set(df_clean_stations["station"])

In [12]:
stations_to_drop = interchange_stations.difference(stations_ref)
stations_to_drop
# These are all London Underground, DLR or otherwise so can be dropped

{'Anerley',
 'Bow Church',
 'Brockley',
 'Bushey',
 'Canary Wharf London Underground',
 'Clapham Junction',
 'Crystal Palace',
 'Elephant & Castle London Underground',
 'Elverson Road',
 'Forest Hill',
 'Gospel Oak',
 'Greenwich',
 'Harringay Green Lanes',
 'Heathrow Terminal  5',
 'Heathrow Terminals 123 London Underground',
 'Heron Quays',
 'Honor Oak Park',
 'Imperial Wharf',
 'Lewisham',
 'Limehouse',
 'New Cross',
 'New Cross Gate',
 'Norwood Junction',
 'Penge West',
 'Poplar',
 'Shadwell',
 'Stratford International',
 'Sydenham',
 'Watford Junction',
 'West Croydon',
 'West India Quay',
 'Westferry',
 'Woolwich Arsenal'}

In [13]:
mask = ~(df_clean_stations["station"].isin(stations_to_drop))
df_interchange_clean = df_clean_stations[mask].reset_index(drop=True)
df_interchange_clean.head(10)

,station,interchange
0,Acton Town,District <> Piccadilly line: 2 minutes
1,Aldgate,Connections between Circle and Metropolitan li...
2,Aldgate East,Connections between District and Hammersmith &...
3,Amersham,Connections between Metropolitan and Chiltern ...
4,Baker Street,Bakerloo to Circle: 8 minutes
5,Baker Street,Bakerloo to Hammersmith & City: 8 minutes
6,Baker Street,Bakerloo to Jubilee: 2 minutes
7,Baker Street,Bakerloo to Metropolitan: 3 minutes
8,Baker Street,Circle to Hammersmith & City: 2 minutes
9,Baker Street,Circle to Jubilee: 8 minutes


Bringing in the cleaned Connection dataset and using the lines in it to extract lines from the Interchange data.

In [14]:
df_connections_ref = pd.read_csv(r"../data/processed/connections_clean.csv")
lines_ref = set(df_connections_ref["Line"])

In [15]:
def retrieve_lines(interchange, lines):
    """
    This function is used to extract underground lines from the "interchange"
    field of the Interchange dataset. It uses a reference set of underground
    lines to extract underground lines from the text, but only if it identifies
    two underground lines in the text.
    """
    
    lines_and_pos = []
    for line in lines:
        idx = interchange.find(line)
        if idx > -1: lines_and_pos.append((line, idx))
    
    lines_and_pos_sorted = sorted(lines_and_pos, key=lambda item: item[1])
    
    if len(lines_and_pos_sorted) == 1:
        return lines_and_pos_sorted[0][0]
    elif len(lines_and_pos_sorted) == 2:
        return lines_and_pos_sorted[0][0], lines_and_pos_sorted[1][0]
    else:
        return np.nan, np.nan

df_interchange_lines = df_interchange_clean.copy(deep=True)
df_interchange_lines[["line_from", "line_to"]] = df_interchange_clean.loc[:, "interchange"].apply(
    lambda x: retrieve_lines(x, lines_ref)
).apply(pd.Series)

df_interchange_lines.head(10)

,station,interchange,line_from,line_to
0,Acton Town,District <> Piccadilly line: 2 minutes,District,Piccadilly
1,Aldgate,Connections between Circle and Metropolitan li...,Circle,Metropolitan
2,Aldgate East,Connections between District and Hammersmith &...,District,Hammersmith & City
3,Amersham,Connections between Metropolitan and Chiltern ...,Metropolitan,NaN
4,Baker Street,Bakerloo to Circle: 8 minutes,Bakerloo,Circle
5,Baker Street,Bakerloo to Hammersmith & City: 8 minutes,Bakerloo,Hammersmith & City
6,Baker Street,Bakerloo to Jubilee: 2 minutes,Bakerloo,Jubilee
7,Baker Street,Bakerloo to Metropolitan: 3 minutes,Bakerloo,Metropolitan
8,Baker Street,Circle to Hammersmith & City: 2 minutes,Circle,Hammersmith & City
9,Baker Street,Circle to Jubilee: 8 minutes,Circle,Jubilee


In [16]:
def retrieve_duration(x):
    idx_colon = x.find(":")
    idx_mins = x.find(" minutes")
    if (idx_colon == -1) or (idx_mins == -1):
        return np.nan
    else:
        return x[(idx_colon+2):idx_mins]   
    
df_interchange_lines = df_interchange_lines.assign(
    duration = df_interchange_lines['interchange'].apply(retrieve_duration)
)

df_interchange_lines.head(10)

,station,interchange,line_from,line_to,duration
0,Acton Town,District <> Piccadilly line: 2 minutes,District,Piccadilly,2
1,Aldgate,Connections between Circle and Metropolitan li...,Circle,Metropolitan,4
2,Aldgate East,Connections between District and Hammersmith &...,District,Hammersmith & City,2
3,Amersham,Connections between Metropolitan and Chiltern ...,Metropolitan,NaN,3
4,Baker Street,Bakerloo to Circle: 8 minutes,Bakerloo,Circle,8
5,Baker Street,Bakerloo to Hammersmith & City: 8 minutes,Bakerloo,Hammersmith & City,8
6,Baker Street,Bakerloo to Jubilee: 2 minutes,Bakerloo,Jubilee,2
7,Baker Street,Bakerloo to Metropolitan: 3 minutes,Bakerloo,Metropolitan,3
8,Baker Street,Circle to Hammersmith & City: 2 minutes,Circle,Hammersmith & City,2
9,Baker Street,Circle to Jubilee: 8 minutes,Circle,Jubilee,8


In [17]:
'''
Dropping entires that have only one line identified. Reasons:
    * The other line is not in the reference set.
    * The interchange is between different branches of a line,
        which isn't being considered at this point.
'''
mask = (
    (df_interchange_lines["line_from"].isna() & df_interchange_lines["line_to"].isna()) |
    (df_interchange_lines["line_from"].notna() & df_interchange_lines["line_to"].notna())
)

df_interchange_lines = df_interchange_lines[mask].reset_index(drop=True)

In [18]:
# Writing the data to csv for review
df_interchange_lines.to_csv(r"../data/raw/interchange_data_raw_v3.csv", encoding="UTF-8")

In [19]:
# Writing the data to another csv that I am going to make maual ammendments too
df_interchange_lines.to_csv(r"../data/raw/interchange_data_raw_v4.csv", encoding="UTF-8")

We now need to use the manually edited file to create two new fields:
* station_from: 'station_name (line)'
* station_to: 'station_name (line)'

This will allow us to then plot the interchange relationships between lines in a station in our graph database.

In [20]:
df_interchange_final = pd.read_csv(r"../data/raw/interchange_data_raw_v4.csv", index_col=0).drop(columns="interchange")
df_interchange_final.head(10)

,station,line_from,line_to,duration
0,Acton Town,District,Piccadilly,2.0
1,Aldgate,Circle,Metropolitan,4.0
2,Aldgate East,District,Hammersmith & City,2.0
3,Baker Street,Bakerloo,Circle,8.0
4,Baker Street,Bakerloo,Hammersmith & City,8.0
5,Baker Street,Bakerloo,Jubilee,2.0
6,Baker Street,Bakerloo,Metropolitan,3.0
7,Baker Street,Circle,Hammersmith & City,2.0
8,Baker Street,Circle,Jubilee,8.0
9,Baker Street,Circle,Metropolitan,8.0


In [23]:
from common_processing import add_line_to_station

df_interchange_final = df_interchange_final.assign(
    station_from = df_interchange_final.apply(lambda row: add_line_to_station(row["station"], row["line_from"]), axis=1),
    station_to = df_interchange_final.apply(lambda row: add_line_to_station(row["station"], row["line_to"]), axis=1)
)

df_interchange_final.head()

,station,line_from,line_to,duration,station_from,station_to
0,Acton Town,District,Piccadilly,2.0,Acton Town (District),Acton Town (Piccadilly)
1,Aldgate,Circle,Metropolitan,4.0,Aldgate (Circle),Aldgate (Metropolitan)
2,Aldgate East,District,Hammersmith & City,2.0,Aldgate East (District),Aldgate East (Hammersmith & City)
3,Baker Street,Bakerloo,Circle,8.0,Baker Street (Bakerloo),Baker Street (Circle)
4,Baker Street,Bakerloo,Hammersmith & City,8.0,Baker Street (Bakerloo),Baker Street (Hammersmith & City)


In [24]:
df_interchange_final = df_interchange_final[["station_from", "station_to", "duration"]]
df_interchange_final.head()

,station_from,station_to,duration
0,Acton Town (District),Acton Town (Piccadilly),2.0
1,Aldgate (Circle),Aldgate (Metropolitan),4.0
2,Aldgate East (District),Aldgate East (Hammersmith & City),2.0
3,Baker Street (Bakerloo),Baker Street (Circle),8.0
4,Baker Street (Bakerloo),Baker Street (Hammersmith & City),8.0


In [25]:
# Saving to processed folder
df_interchange_final.to_csv(r"../data/processed/interchange_clean.csv")